In [1]:
from fastai.vision import *
from fastai.metrics import error_rate

In [2]:
import torch
print(torch.cuda.get_device_name(0))

GeForce GTX 1050 Ti


In [3]:
base_dir = 'data/'
path = Path(base_dir)
path.ls()

[WindowsPath('data/.ipynb_checkpoints'),
 WindowsPath('data/models'),
 WindowsPath('data/sample_submission.csv'),
 WindowsPath('data/test'),
 WindowsPath('data/train'),
 WindowsPath('data/train_labels.csv')]

## Data Augmentation


In [4]:
tfms = get_transforms(do_flip=True,
                     flip_vert=True,
                     max_rotate=30.0,
                     p_affine=0.80,
                     p_lighting=0.80)

In [5]:
base_dir = 'data/'

bs=64 # also the default batch size
data = ImageDataBunch.from_csv(
    base_dir, 
    ds_tfms=tfms,
    valid_pct=0.05,
    size=224, 
    suffix=".tif",
    folder="train", 
    test="test",
    csv_labels="train_labels.csv", 
    bs=bs)

In [6]:
data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (209024 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,1,0,0,0
Path: data;

Valid: LabelList (11001 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,1,0,0,0
Path: data;

Test: LabelList (57458 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: data

In [7]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate, callback_fns=ShowGraph).to_fp16()

In [8]:
learn.load('fastai_freeze_model_01')

Learner(data=ImageDataBunch;

Train: LabelList (209024 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,1,0,0,0
Path: data;

Valid: LabelList (11001 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,1,0,0,0
Path: data;

Test: LabelList (57458 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: data, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1,

## The Accuracy of Using Single ResNet50 Model

In [9]:
preds,y, loss = learn.get_preds(with_loss=True)

# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc*100))


The accuracy is 96.13671112060547 %.


In [10]:
num_model = 5
model_list = []

for i in range(num_model):
    idx = i
    modelname = "fastai_freeze_model_0" + str(idx+1)
    print("load", modelname)
    _learn = cnn_learner(data, models.resnet50, metrics=error_rate, callback_fns=ShowGraph).to_fp16()
    model_list.append(_learn)
    model_list[i].load(modelname)
print("Done")

load fastai_freeze_model_01
load fastai_freeze_model_02
load fastai_freeze_model_03
load fastai_freeze_model_04
load fastai_freeze_model_05
Done


## Test Time Augmentation

In [11]:
# Augmentation for Testing Data
data.test_ds.tfms = data.train_ds.tfms

num_TTA = 4 
results = []
for i in range(num_TTA):
    for j in range(num_model):
        _preds, y = model_list[j].get_preds()
        results.append(_preds.data.numpy())

## Ensemble Learning — Bagging the Results

In [20]:
print((results[0][]))

(11001, 2)


In [24]:
num_result = len(results)
num_sample = results[0].shape[0]
print(num_result)
print(num_sample)

bagging_results = np.zeros((num_sample, 2))
for i in range(num_sample):
    for j in range(num_result):
        bagging_results[i] += results[j][i]

bagging_results = bagging_results / num_result    

20
11001


In [30]:
acc = accuracy(tensor(bagging_results), y)
print('The accuracy is {0} %.'.format(acc*100))

The accuracy is 98.6728515625 %.
